In [1]:
import numpy as np
import pandas as pd


In [2]:

target_df = pd.read_excel(r"../data\Raw\Target_PER1p5_nacres_fe_database_v1-0-2023.xlsx", sheet_name="NACRES-EF")
target_df["nacres.description.en"] = target_df["nacres.description.en"].str.rstrip()
#OPen data\Raw\nacres-janvier-2022.xls
nacres_df = pd.read_excel(r"../data\Raw\nacres-janvier-2022.xls", sheet_name="M9 NACRES")

c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
#rename columns nacres_df["Libellé étendu nouvelle nomenclature achat"] to "nacres.description.en" and "Codes Nacres" to "nacres.code"
nacres_df.rename(columns={"Codes Nacres": "nacres.code"}, inplace=True)
nacres_df.rename(columns={"Libellé étendu nouvelle nomenclature achat": "nacres.description.en"}, inplace=True)

#For each nacres_df["Codes Nacres"] that has a length of less than 5, append these rows to target_df
nacres_df["nacres.code"] = nacres_df["nacres.code"].astype(str)
nacres_df["nacres.code"] = nacres_df["nacres.code"].str.strip()
nacres_df["nacres.description.en"] = nacres_df["nacres.description.en"].str.rstrip()
#append rows to target_df, without using append
nacres_df_domain = nacres_df[nacres_df["nacres.code"].str.len() < 5]


In [4]:
nacres_df_domain = nacres_df_domain[['nacres.code', 'nacres.description.en']]

In [1]:
import numpy as np
import pandas as pd
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
from transformers import MarianMTModel, MarianTokenizer
import torch

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Declare global variables for the model and tokenizer
model = None
tokenizer = None

def load_model_and_tokenizer(model_name="Helsinki-NLP/opus-mt-fr-en"):
    global model, tokenizer
    if model is None or tokenizer is None:
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)
        if torch.cuda.is_available():
            model = model.to('cuda')
    return model, tokenizer

def translate_batch(texts):
    model, tokenizer = load_model_and_tokenizer()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    encoded_inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        translated_tokens = model.generate(**encoded_inputs)
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
    return translated_texts

def process_chunk(chunk, columns_to_translate, suffix):
    try:
        for column in columns_to_translate:
            chunk_size = 10
            translated_texts = []
            for i in range(0, len(chunk), chunk_size):
                batch_texts = chunk[column].iloc[i:i + chunk_size].tolist()
                translated_texts.extend(translate_batch(batch_texts))
            chunk[column + suffix] = translated_texts
        return chunk
    except Exception as e:
        logging.error(f"Error processing chunk: {e}")
        raise

def translate(df, columns_to_translate, suffix="_translated"):
    num_splits = min(10, len(df))  # Optimize number of splits
    chunks = np.array_split(df, num_splits)
    with ProcessPoolExecutor(max_workers=num_splits) as executor:
        futures = []
        for idx, chunk in enumerate(chunks):
            logging.info(f"Submitting chunk {idx + 1}/{num_splits} for processing")
            futures.append(executor.submit(process_chunk, chunk, columns_to_translate, suffix))
        processed_chunks = []
        for idx, future in enumerate(as_completed(futures)):
            try:
                result = future.result()
                logging.info(f"Chunk {idx + 1}/{num_splits} processed")
                processed_chunks.append(result)
            except Exception as e:
                logging.error(f"Error in future result: {e}")
    translated_df = pd.concat(processed_chunks)
    return translated_df

def translate_DB(input_data, columns_to_translate=None, output_file_name=None):
    if isinstance(input_data, str):
        # Handle string input
        translated_text = translate_batch([input_data])[0]
        result = {columns_to_translate[0]: input_data, f"{columns_to_translate[0]}_translated": translated_text}
        return pd.DataFrame([result])

    elif isinstance(input_data, pd.DataFrame):
        df = input_data
    else:
        raise ValueError("Input must be a string or a DataFrame")

    if not columns_to_translate:
        logging.warning("No columns to translate were provided, no translation will be performed.")
        return df

    translated_df = translate(df, columns_to_translate, suffix="_translated")
    for column in columns_to_translate:
        translated_df[f"{column} original"] = translated_df[column]
        translated_df[column] = translated_df[f"{column}_translated"]
        translated_df.drop(columns=[f"{column}_translated"], inplace=True)

    if output_file_name:
        translated_df.to_pickle(output_file_name, index=False)

    return translated_df


c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
nacres_df_domain[:4]

,nacres.code,nacres.description.en
0,A,APPROVISIONNEMENTS GENERAUX
1,AA,ALIMENTATION - RESTAURATION - HOTELLERIE
2,AA.0,PRODUITS ALIMENTAIRES CONGELES
8,AA.1,PRODUITS ALIMENTAIRES FRAIS


In [9]:
nacres_df_domain_translated = translate_DB(nacres_df_domain[:4], ['nacres.description.en'])

c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [17]:
target_df_2 = pd.concat([target_df, nacres_df_domain_translated], ignore_index=True)

In [19]:


import pandas as pd
import numpy as np


# Function to replace NaNs with medians in multiple float columns
def replace_nan_with_median(df):
    float_columns = df.select_dtypes(include=['float64']).columns

    def get_median_for_subcategories(code, column):
        subcategory_mask = df['nacres.code'].str.startswith(code) & (df['nacres.code'].str.len() > len(code))
        subcategory_values = df.loc[subcategory_mask, column].values
        if len(subcategory_values) > 0:
            median_value = np.nanmedian(subcategory_values)
        return median_value
    
    for column in float_columns:
        for i, row in df.iterrows():
            code = row['nacres.code']
            if len(code) < 5 and pd.isna(row[column]):
                median_value = get_median_for_subcategories(code, column)
                df.at[i, column] = median_value

    return df

# Replace NaNs with the calculated medians
df = replace_nan_with_median(target_df_2)

c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\cruz\AppDat

In [20]:
df

,nacres.code,nacres.description.fr,nacres.description.en,ceda.code,ceda.description,ceda.ef.kg.co2e.per.euro,ceda.uncertainty.attr.kg.co2e.per.euro,ceda.uncertainty.80pct.kg.co2e.per.euro,ademe.code,ademe.description,...,micro.uncertainty.kg.co2e.per.euro,method,per1p5.ef.kg.co2e.per.euro,per1p5.uncertainty.attr.kg.co2e.per.euro,per1p5.uncertainty.80pct.kg.co2e.per.euro,uncertainty.groupby,module,category,scope.source.v4,scope.source.v5
0,AA01,"PAINS, PATISSERIES, VIENNOISERIES CONGELES","FROZEN BREADS, PASTRIES, PASTRIES",NaN,NaN,NaN,NaN,NaN,ADEME14,Produit agro-alimentaires transformes,...,NaN,macro,1.10,0.10,0.880,AA05,purchases,lab.life,9.0,4.1
1,AA02,PRODUITS CARNES CONGELES,FROZEN MEAT PRODUCTS,NaN,NaN,NaN,NaN,NaN,ADEME13,Produits agricoles et de la mer,...,NaN,macro,1.50,0.70,1.200,AA04,purchases,lab.life,9.0,4.1
2,AA03,PRODUITS DE LA MER OU DEAU DOUCE CONGELES,FROZEN SEA OR FRESH WATER PRODUCTS,NaN,NaN,NaN,NaN,NaN,ADEME13,Produits agricoles et de la mer,...,NaN,macro,1.50,0.70,1.200,AA04,purchases,lab.life,9.0,4.1
3,AA04,FRUITS ET LEGUMES CONGELES,FROZEN FRUITS AND VEGETABLES,NaN,NaN,NaN,NaN,NaN,ADEME13,Produits agricoles et de la mer,...,NaN,macro,1.50,0.70,1.200,AA04,purchases,lab.life,9.0,4.1
4,AA05,PREPARATIONS ALIMENTAIRES ET PLATS CUISINES CO...,FROZEN FOOD PREPARATIONS AND DISHES,NaN,NaN,NaN,NaN,NaN,ADEME14,Produit agro-alimentaires transformes,...,NaN,macro,1.10,0.10,0.880,AA05,purchases,lab.life,9.0,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,XE.0,NaN,AVANCES ET ACOMPTE SUR IMMOBILISATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,XE.1,NaN,ACQUISITIONS IMMOBILISATIONS INCORPORELLES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016,XE.2,NaN,ACQUISITIONS IMMOBILISATIONS CORPORELLES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,XF,NaN,FACTURATION INTERNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.34,0.33,0.272,NaN,NaN,NaN,9.0,4.2
